In [ ]:
from cryptography.hazmat.primitives import hashes

class MerkleTree():

    class Node():
        def __init__(self, parent = None, right = None, left = None):
            self.item = None
            self.parent = parent
            self.right = right
            self.left = left
        
        def get_parent(self):
                return self.parent  
             
        def get_right(self):
                return self.right
                
        def get_left(self):
                return self.left
        
        def _set_parent(self, node):
            if isinstance(node, type(self)):
                 self.parent = node
                
        def _set_right(self, node):
            if isinstance(node, type(self)):
                 self.right = node  
        
        def _set_left(self, node):
            if isinstance(node, type(self)):
                 self.left = node
            
    def __init__(self, text):
        self.text = text
        self.digest = hashes.Hash(hashes.SHA256())
        self.merkle_tree = self._build_tree(self.text)
         

    
    def _build_tree(self, text):

        tree_layers = []
        last_layer = []

        """Decompose string in div chars per node"""
        for node in self._decompose_string(text, div = 4): last_layer.append(node)

        if len(last_layer) == 0: return tree_layers
        
        tree_layers.append(last_layer)
        
        remaining_layers = 1
        while remaining_layers > 0:
            new_layer, remaining_layers = self._build_layer(last_layer) 
            tree_layers.append(new_layer)
            last_layer=new_layer
        
        return tree_layers


    def _decompose_string(self, text, div):
        l = len(self.text)
        n_node = int(l/div) + (int(1) if l % div != 0 else 0)

        for i in range(n_node):
            start_seq = i*div
            node = self.Node()
            node.item = text[start_seq:start_seq+div]
            yield node

    def _build_layer(self, last_layer):
        
        len_last_layer= len(last_layer)
        residual = len_last_layer%2
        n_node = int(len_last_layer/2) + residual
        layer = []
        j = 0

        for i in range(n_node):
            node = self.Node()

            first_node = last_layer[j]
            len_last_layer -= 1;    j += 1
            second_node = (last_layer[j] if len_last_layer > 0 else None)
            len_last_layer -= 1;    j += 1 

            self._link_nodes(node, first_node, second_node)

            layer.append(node)

        return layer, len(layer)-1
    

    def _link_nodes(self, node, first, second):
        if first is not None:
            node._set_left(first)
            first._set_parent(node)
            self.digest.update(node.get_left().item.encode())
        if second is not None:
            node._set_right(second)
            second._set_parent(node)
            self.digest.update(node.get_right().item.encode())
        
        node.item = self.digest.finalize().hex()
        self.digest = hashes.Hash(hashes.SHA256())

    



my_MrkTree = MerkleTree("The integrity of this message is essential!")

print(my_MrkTree.text)
print(my_MrkTree.merkle_tree[-1][0].item) # Merkle Root

Hello World!  Michele
e33ff3cf50766869830978230ff2d7260f34c1aa3b5c65a60abb6364fbea3e6f
